# 🚀 NurseSim-Triage: Cloud Deployment (Direct Upload)

**Push your validated Local Model AND Model Card to Hugging Face Hub**

This notebook takes the robustly trained model from **Google Drive** and uploads the LoRA adapter to your Hugging Face repository using the `huggingface_hub` API.

### 📋 Target Repository
- **Repo ID:** `NurseCitizenDeveloper/NurseSim-Triage-Llama-3.2-3B`
- **Source:** `/content/drive/MyDrive/nursesim_lora_llama3_robust`

In [ ]:
%%capture
!pip install huggingface_hub

In [ ]:
import os
from google.colab import drive
from huggingface_hub import login, HfApi

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Login to Hugging Face
from google.colab import userdata
try:
    login(token=userdata.get('HF_TOKEN'))
except:
    print("⚠️ HF_TOKEN not found in secrets. Please login manually.")
    login()

In [ ]:
# 3. Create Updated Model Card (README.md)
readme_content = """---
title: NurseSim Triage
emoji: 🏥
colorFrom: blue
colorTo: indigo
sdk: docker
pinned: false
tags:
- healthcare
- triage
- clinical-nlp
- reinforcement-learning
---

# NurseSim-RL: A Healthcare Agent Environment for Clinical Triage

[![AgentBeats A2A](https://img.shields.io/badge/AgentBeats-A2A%20Enabled-purple)](https://agentbeats.dev/ClinyQAi/nursesim-triage)
[![OpenEnv Challenge](https://img.shields.io/badge/OpenEnv-Challenge%202026-blue)](https://rdi.berkeley.edu/agentx-agentbeats)
[![Hugging Face Model](https://img.shields.io/badge/🤗-Model-yellow)](https://huggingface.co/NurseCitizenDeveloper/NurseSim-Triage-Llama-3.2-3B)

> **OpenEnv Challenge Entry** | Berkeley RDI AgentX-AgentBeats Competition  
> A Gymnasium-compatible RL environment for training AI agents to perform clinical triage using the Manchester Triage System (MTS).

![NurseSim Demo](https://huggingface.co/spaces/NurseCitizenDeveloper/NurseSim-Triage-Demo/raw/main/docs/demo.gif)

## 🎯 Overview

**NurseSim-RL** simulates the decision-making process of a Triage Nurse in an Accident & Emergency (A&E) department. The agent must assess patients based on their chief complaint and vital signs, then assign an appropriate triage category (1-5) according to the Manchester Triage System.

### Key Features
- **Gymnasium-Compatible:** Standard RL interface for easy integration.
- **Expanded Dataset:** Trained on **2,100+** synthetic patient archetypes across all 5 MTS categories.
- **Safety-Aware Rewards:** Heavy penalties for under-triaging critical patients.
- **Fine-Tuned Agent:** Llama 3.2 3B trained with Unsloth (4-bit QLoRA).
- **A2A Protocol:** Agent-to-Agent evaluation via AgentBeats platform.

## 📊 Training Results & Validation

The agent was fine-tuned using **Unsloth** on a Llama 3.2 3B base model with an expanded dataset of ~2,100 clinical scenarios.

### ✅ Performance Metrics (Local Validation)
Evaluated on 15 Gold-Standard Clinical Scenarios using GPT-5.2 as a Clinical Judge.

| Metric | Value | Description |
|--------|-------|-------------|
| **Accuracy** | **60%** | Exact match with Manchester Triage Categories (1-5) |
| **Safety** | **70%+** | Pass Rate for critical life-threat detection (Sepsis, Anaphylaxis) |
| **Latency** | < 2s | Inference time on T4 GPU |

### 🧠 Key Methodology: Age-Adjusted Triage
Our validation revealed that **parsing Age and Gender** from the patient description is critical for accurate risk stratification (e.g., separating "Chest Pain" in a 72M vs 20M). The model effectively learned these demographic risk factors.

## 🩺 Clinical Framework: Manchester Triage System

| Category | Priority | Target Time | Example |
|----------|----------|-------------|---------|
| 1 | Immediate | 0 min | Cardiac arrest, Anaphylaxis |
| 2 | Very Urgent | 10 min | Chest pain, Stroke |
| 3 | Urgent | 60 min | Abdominal pain, Fractures |
| 4 | Standard | 120 min | Minor injuries, Mild illness |
| 5 | Non-Urgent | 240 min | Minor cuts, GP-suitable |

## 📚 Resources

- **Live Demo:** [Hugging Face Space](https://huggingface.co/spaces/NurseCitizenDeveloper/NurseSim-Triage-Demo)
- **Model Weights:** [NurseSim-Triage-Llama-3.2-3B](https://huggingface.co/NurseCitizenDeveloper/NurseSim-Triage-Llama-3.2-3B)
- **Training Report:** [W&B Dashboard](https://wandb.ai/mrlincs-nursing-citizen-development/huggingface)

## 🙏 Acknowledgements

Built for the **OpenEnv Challenge 2026** by the **NurseCitizenDeveloper** team.
Special thanks to **Clare Cable**, **Joanne Bosanquet**, and the **Burdett Trust for Nursing** for their leadership in clinical innovation.
"""

with open("README.md", "w") as f:
    f.write(readme_content)

print("✅ Generated updated README.md locally.")

In [ ]:
# 4. Upload to Hugging Face
adapter_path = "/content/drive/MyDrive/nursesim_lora_llama3_robust"
repo_id = "NurseCitizenDeveloper/NurseSim-Triage-Llama-3.2-3B"

print(f"🔄 Verifying Validated Model at: {adapter_path}...")
if not os.path.exists(adapter_path):
    raise FileNotFoundError(f"❌ Path not found. Check Drive!")

if not os.path.exists("README.md"):
    raise FileNotFoundError("❌ README.md not found. Did you run the cell above?")

api = HfApi()

print(f"🚀 Checking Repository: {repo_id}...")
try:
    # Create repo if it doesn't exist (Handling case where user deleted it)
    api.create_repo(repo_id=repo_id, repo_type="model", exist_ok=True)
    print(f"   ✅ Repository ready.")

    print(f"🚀 Uploading Model & Card...")

    # 1. Upload Model Folder
    api.upload_folder(
        folder_path=adapter_path,
        repo_id=repo_id,
        repo_type="model",
        commit_message="Upload robust trained LoRA adapter (Acc 60%)"
    )
    print("   ✅ Model weights uploaded.")
    
    # 2. Upload README
    api.upload_file(
        path_or_fileobj="README.md",
        path_in_repo="README.md",
        repo_id=repo_id,
        repo_type="model",
        commit_message="Update Model Card with Validation Results"
    )
    print("   ✅ Model Card updated.")
    
    print("\n🎉 FULL DEPLOYMENT COMPLETE!")
    print(f"🔗 Check it out: https://huggingface.co/{repo_id}")

except Exception as e:
    print(f"❌ Upload failed: {e}")